In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import display, HTML, SVG

In [2]:
from sklearn.metrics import auc, f1_score, precision_recall_curve, roc_curve, accuracy_score, roc_auc_score, average_precision_score, precision_score, recall_score
from sklearn.metrics import auc, precision_recall_curve, roc_curve, confusion_matrix, roc_auc_score, roc_curve, f1_score, accuracy_score
import pandas
import numpy as np
import json
import matplotlib.pyplot as plt
import os
import ast
import seaborn
from sklearn.calibration import calibration_curve
from scipy.stats import binom_test, fisher_exact, chi2_contingency

from db import Result
import math
import papermill as pm
from vis.utils.utils import load_img
from tabulate import tabulate
import svgutils.transform as sg

from config import config
pandas.options.display.float_format = '{:,.3f}'.format

Using TensorFlow backend.
/home/intrepidlemon/.local/share/virtualenvs/deep-ultrasound-eb7oaR1r/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/intrepidlemon/.local/share/virtualenvs/deep-ultrasound-eb7oaR1r/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/intrepidlemon/.local/share/virtualenvs/deep-ultrasound-eb7oaR1r/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be underst

In [3]:
from evaluate import load, get_results, get_labels, adjusted_wald, transform_binary_probabilities, transform_binary_predictions, calculate_accuracy_loss, plot_confusion_matrix, plot_precision_recall, plot_roc_curve, calculate_pr_auc, calculate_confusion_matrix_stats, calculate_confusion_matrix, plot_tsne, accession_from_filename, plot_expert_confusion, plot_grad_cam, get_expert_results

In [4]:
from data import describe, all_features, data, print_describe, load_single
from run import characterize_data

In [5]:
from collections import defaultdict

# All models

In [6]:
DIRECTORY = os.path.join(os.environ["DATA_DIR"], "liver-ultrasound")
CROPS = ["free", "fixed", "uncertain-free", "uncertain-fixed"]
DESCRIPTIONS = {
    "free": "free-0",
    "fixed": "fixed-0", 
    "uncertain-free": "c3-c4-free-0",
    "uncertain-fixed": "c3-c4-fixed-0", 
}
DATA_FOLDERS = {
    ("free"): os.path.join(DIRECTORY, "free"), 
    ("fixed"): os.path.join(DIRECTORY, "fixed"), 
    ("uncertain-free"): os.path.join(DIRECTORY, "c3-c4-free"), 
    ("uncertain-fixed"): os.path.join(DIRECTORY, "c3-c4-fixed"), 
}
MODALITIES = CROPS
MODALITY_KEY = {
    "free": "Free - Complete",
    "fixed": "Fixed - Complete", 
    "uncertain-free": "Free - Uncertain",
    "uncertain-fixed": "Fixed - Uncertain", 
}

# Evaluate all models

In [7]:
CROPS = ["free", "fixed"]
SUBSETS = ["complete", "uncertain"]
UUIDS = {
    ("free", "complete"): "04cfb42d-3e0b-4516-aa85-56f19cb4d6b9",
    ("fixed", "complete"): "cbd87d38-62c5-4196-9c5f-d967eac37085", 
    ("free", "uncertain"): "9eecc121-a682-4be8-9aa1-f43f4a8b69a7",
    ("fixed", "uncertain"): "fd0391b6-3c63-48d9-a0b4-dc0923f48c32",
}
DATA_FOLDERS = {
    ("free", "complete"): os.path.join(DIRECTORY, "free"), 
    ("fixed", "complete"): os.path.join(DIRECTORY, "fixed"), 
    ("free", "uncertain"): os.path.join(DIRECTORY, "c3-c4-free"), 
    ("fixed", "uncertain"): os.path.join(DIRECTORY, "c3-c4-fixed"), 
}

In [8]:
for crop in CROPS: 
    for subset in SUBSETS:
        pm.execute_notebook(
            "evaluate-specific-model.ipynb",
            "evaluate-{}-{}.ipynb".format(crop, subset),
            parameters = dict(
                DIRECTORY = DATA_FOLDERS[(crop, subset)],
                MODEL_UUID = UUIDS[(crop, subset)],
                MODEL_TYPE = "v2",
                SELECTIONS = "[]", 
                PARAMETERS_ID = "{}-{}".format(crop, subset),
            ),
        )

In [26]:
def four_group(basename="figures/accuracy-by-category-{}-{}.svg", X1=70, Y1=10, X2=460, Y2=290, width="750pt", height="450pt"):
    figures = list()
    plots = list()
    fig = sg.SVGFigure(width, height)
    movements = {
        ("free", "complete"): (X1, Y1),
        ("fixed", "complete"): (X1, Y2), 
        ("free", "uncertain"): (X2, Y1),
        ("fixed", "uncertain"): (X2, Y2), 
    }
    for crop in CROPS: 
        for subset in SUBSETS:
            f = sg.fromfile(basename.format(crop, subset))
            figures.append(f)
            plot = f.getroot()
            plot.moveto(movements[(crop, subset)][0], movements[(crop, subset)][1])
            plots.append(plot)
            fig.append([plot])
    txt1 = sg.TextElement(0, Y2/2, "free crop", size=12, weight="bold")
    txt2 = sg.TextElement(0, Y2 + Y2/2, "fixed crop", size=12, weight="bold")
    txt3 = sg.TextElement(X2/2, 12, "complete", size=12, weight="bold")
    txt4 = sg.TextElement(X2 + X2/5,12, "uncertain subgroup", size=12, weight="bold")
    fig.append([txt1, txt2, txt3, txt4])
    fig.save(basename.format("all", "all"))
    return SVG(basename.format("all", "all"))

In [42]:
def two_group(svg1, svg2, savename, X1=70, Y1=30, X2=460, Y2=290, width="750pt", height="450pt"):
    figures = list()
    plots = list()
    fig = sg.SVGFigure(width, height)
    f = sg.fromfile(svg1)
    figures.append(f)
    plot = f.getroot()
    plot.moveto(X1, Y1)
    plots.append(plot)
    fig.append([plot])
    f = sg.fromfile(svg2)
    figures.append(f)
    plot = f.getroot()
    plot.moveto(X1, Y2)
    plots.append(plot)
    fig.append([plot]) 
    txt1 = sg.TextElement(0, Y1, "A", size=24, weight="bold")
    txt2 = sg.TextElement(0, Y2, "B", size=24, weight="bold")
    fig.append([txt1, txt2])
    fig.save(savename)
    return SVG(savename)

In [37]:
fig = four_group("figures/{}-{}-confusion-matrix.svg", X2=390)

In [38]:
fig = four_group("figures/{}-{}-confusion-matrix-expert-1.svg", X2=390)

In [55]:
two_group(
    "figures/model-free-complete-tsne.svg", 
    "figures/label-free-complete-tsne.svg", 
    "figures/combined-free-complete-tsne.svg"
)

In [49]:
two_group(
    "figures/complete-test-roc.svg", 
    "figures/uncertain-test-roc.svg", 
    "figures/combined-test-roc.svg",
)

In [15]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')